# Final Exam - Open

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# Instructions

## Please note the following:

Points
* You have `100` minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo - you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

- For Section 1:
    * No data file is needed.
- for Section 2:
    * `data/treasury_quotes_clean_2024-12-31.xlsx`, sheet `selected quotes`.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 70     |
| 2       | 30     |

***

## Submitting your Exam

Submission
* You will upload your solution to the `Exam - Open` assignment on Canvas. 
* Submit a compressed, "zipped", folder containing all code according to the file structure below.
* Name your submitted, zipped, folder `exam-open-LASTNAME-FIRSTNAME.zip`.
* Be sure to **submit** on Canvas, not just **save** on Canvas.

Your submission should **include all code and data used in your analysis** in the following folder structure.
* We strongly prefer all submissions are structred this way, and it will improve grading accuracy for partial credit. 
* Still, if you're struggling to get this working in time, no worries; just structure as comfortable and submit **everything used** for your submission.

__Exam Submission Structure:__

```plaintext
exam-open-LASTNAME-FIRSTNAME.zip/
│── exam-open.ipynb
│── data/
│   ├── example_data.csv
│── modules/
│   ├── my_functions.py

### Validating your folder structure

The next cell tests that you have this folder structure implemented.

In [ ]:
from pathlib import Path
import sys
import pandas as pd

# Get the directory of the notebook (assumes Jupyter Notebook is always used)
BASE_DIR = Path().resolve()

# Define paths for data and modules
DATA_DIR = BASE_DIR / "data"
MODULES_DIR = BASE_DIR / "modules"

# Check if expected directories exist
if not DATA_DIR.exists():
    print(f"Warning: Data directory '{DATA_DIR}' not found. Check your file structure.")

if not MODULES_DIR.exists():
    print(f"Warning: Modules directory '{MODULES_DIR}' not found. Check your file structure.")

# Ensure Python can import from the modules directory
if str(MODULES_DIR) not in sys.path:
    sys.path.append(str(MODULES_DIR))

# Load exam data
EXAMPLE_DATA_PATH = DATA_DIR / "example_data.csv"

if EXAMPLE_DATA_PATH.exists():
    example_data = pd.read_csv(EXAMPLE_DATA_PATH)
else:
    print(f"Warning: '{EXAMPLE_DATA_PATH.name}' not found. Ensure it's in the correct directory.")


***

# 1. The "Steepener" Trade

A popular trade in 2024 was the 2s10s "steepener" trade, which was a trade designed to capture the changes in the slope of yield curve while being hedged to parallel shifts in it. In particular, the trade believes that the 2-year treasury yield is too high relative to the 10-year treasury yield. We will replicate this trade.

`Initial date:` July 1st, 2024 (corresponding to $t=0$)

__2-year Bond:__
- `Time To Maturity:` 2 years
- `Coupon Frequency:` semi-annual
- `Coupon Rate:` 4%
- `Annualized semi-annual YTM:` 4.74%

__10-year Bond:__
- `Time To Maturity:` 10 years
- `Coupon Frequency:` semi-annual
- `Coupon Rate:` 2%
- `Annualized semi-annual YTM:` 4.44%

__Note__: Assume that coupons are paid exactly at the $t=0.5, 1, 1.5, ...$ year marks. 

### 1.1. (5pts)
Based on the given information, report the dirty price of each bond as of $t=0$ assuming a face value of $100. Display your answer rounded to 4 decimal places.

### 1.2. (5pts)

Report the modified duration and modified-dollar-duration for each bond. Calculate these modified duration values with respect to changes in the quoted semi-annual YTM. Display your answer rounded to 4 decimal places.

### 1.3. (15pts)
We want to construct our portfolio in a modified-dollar-duration neutral fashion such that we are well hedged to parallel moves in the yield curve. 

Using the results from `1.2`, construct a portfolio aligned with the trade thesis that the YTM spread will converge, and go long one bond and short the other. 

Size the portfolio.
* Your position has a $1,000 (modified duration) exposure to a symmetric 1 basis-point convergence in the YTM spread.
* Assume the spread converges evenly, with 0.5bp convergence on each side.
* We assume no leverage is used. 

Report your position in each bond as:
* the notional amount.
* the market value.
* the number of bond contracts (assuming each contract has a face/par value of $100).

Display your answers rounded to the nearest integer.

### 1.4. (10pts)

Assume we are now at $t=0.05$, and the bond yields have now moved.

__2-year Bond:__
- `Annualized semi-annual YTM:` 4.32%

__10-year Bond:__
- `Annualized semi-annual YTM:` 4.14%

For each bond, report the new dirty price and clean price. Display your answer rounded to 4 decimal places.

### 1.5. (10pts)

Still at $t=0.05$, report the profit-and-loss **(PnL)** of each position, and also for the entire portfolio, using:
* Duration (first-order YTM changes).

Use the $t=0$ modified-dollar-duration values from part `1.2`. 

Display your answer rounded to 4 decimal places.

### 1.6 (10pts)

Repeat `1.5`, but this time uapproximating the PnL using
* Duration + convexity

### 1.7. (5pts)

Repeat `1.5`, but this time calculating the PnL via
* full repricing (using your pricing results from part `1.4`.)

Display the comparison of the PnL across the three methods of `1.5`, `1.6`, and `1.7`.

### 1.8 (5 pts)

* What do the results from `1.6` capture that `1.5` does not, and why might this be important to consider?
* For any change in the YTM spread, do we expect the first order P/L estimate to always underestimate or overestimate the second order P/L estimate? Explain your reasoning.

### 1.9 (5pts)

Name two things the results from `1.7` capture that `1.6` does not.

***

# 2. Extracting Curves

Use the selected treasury quotes in the file, `treasury_quotes_clean_2024-12-31.xlsx`, using sheet `selected quotes`.

* Market quotes are provided for treasury securities expiring approximately every six months over the next `7` years.
* You are welcome to assume that they mature at exaclty six-year increments or to use the exact dates.
* Consider the `price` column to represent the dirty mid price.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
filepath = '../data/treasury_quotes_clean_2024-12-31.xlsx'
SHEET = 'selected quotes'
quotes = pd.read_excel(filepath,sheet_name=SHEET).set_index('KYTREASNO')
quotes

### 2.1. (15pts)

Bootstrap the spot curve. 

For maturity `T=5`, report
* the discount factor
* the semiannually-compounded discount rate
* the continuously-compounded discount rate

Round your answers to four decimal places.

### 2.2. (10pts)

Use the bootstrapped spot curves to calculate the forward curves.

For maturity $T_1=5 \rightarrow T_2=5.5$, report
* the forward discount factor.
* the semiannually-compounded forward rate.
* the continuously-compounded forward rate.

Round your answers to four decimal places.

### 2.3. (5pts)

For which maturity ($T_1$ and $T_2$) is the forward rate highest?

***